In [1]:
import itertools
import sys
import time

sys.path.append(".././")
# sys.path.append(".././benchmarks")
sys.path.append("../../blackjax")
import os

import jax
import jax.numpy as jnp
import blackjax

os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=" + str(128)
num_cores = jax.local_device_count()

from metrics import benchmark
from benchmarks.lattice import Phi4
from benchmarks.sampling_algorithms import (

    adjusted_hmc,
    adjusted_mclmc,
    adjusted_mclmc_no_tuning,
    adjusted_mclmc_tuning,
    nuts,
    unadjusted_mclmc,
    unadjusted_mclmc_no_tuning,
    unadjusted_mclmc_tuning,
    unadjusted_underdamped_langevin,
    unadjusted_underdamped_langevin_no_tuning,
)
from benchmarks.inference_models import (
    Banana,
    Brownian,
    Funnel,
    Gaussian,
    GermanCredit,
    Rosenbrock,
    StochasticVolatility,
)

# model = Gaussian(ndims=10,condition_number=100)
# model = GermanCredit()
# model = Phi4()
# model = StochasticVolatility()
# model = Gaussian(ndims=10)
# model = Rosenbrock()
# model = Funnel()
# model = Brownian()
n = 10000
num_chains = 4

print(Phi4(100,1.0).E_x2 )
# raise Exception

tic = time.time()
ess, ess_avg, ess_corr, params, acceptance_rate, grads_to_low_max, _,bias = benchmark(
    model=Phi4(100,1.0),
    sampler=unadjusted_mclmc(integrator_type="mclachlan", preconditioning=False, num_windows=1,),
    key=jax.random.PRNGKey(1), 
    n=n,
    batch=num_chains, 
    pvmap=jax.pmap 
)
toc = time.time()
print(f"Time elapsed {toc-tic}")

print(f"\nGradient calls for unadjusted MCLMC to reach standardized RMSE of X^2 of 0.1: {grads_to_low_max} (avg over {num_chains} chains and dimensions)")


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/global/u1/r/reubenh/MicroCanonicalHMC/benchmarks/.././benchmarks/lattice.py:35: UserWarning: ground truth for e_x2_100_1.0 not found
  warnings.warn(f"ground truth for e_x2_{self.L}_{self.lam} not found")


./benchmarks/ground_truth/Phi4/e_x2_100_1.0.pkl


AttributeError: 'Phi4' object has no attribute 'E_x2'